## GENERATION OF MARKOV MODEL

**I use the decimal digit of pi to generat a markov Model (Fibonnacci dosen't work the propabilities are to near to be intressting)**

In [52]:
import numpy as np
import pandas as pd
import math

from fractions import Fraction
from collections import defaultdict, Counter
from iteration_utilities import deepflatten #flatten nested lists


from music21 import midi, note, stream, instrument, meter, key
import itertools
import random


import nltk #for NLP
import string
import mchmm #for Markov models

#Just a helper function
def flatten_list(hierarchical_list):
    #Flatten a list of lists
    return [x for sublist in hierarchical_list for x in sublist]

In [58]:
#mettre les decimeaux de pi dans une liste

def dec_pi(nterms) : 
    pi = math.pi * pow(10, nterms-1)
    
    pilist = []
 
    for i in range(nterms) :      
        pilist.append(pi%10) 
        pi/=10
    
    return pilist
     


In [78]:
pi_list = dec_pi(300)
mylist = []

for digit in pi_list :
    if digit%5 == 0 :
        mylist.append(0)
    if digit%5==1 :
        mylist.append(1)
    if digit%5 == 2 :
        mylist.append(2)
    if digit%5 == 3 :
        mylist.append(3)
    if digit%5 == 4 : 
        mylist.append(4)

In [79]:
#built the bigram

bigrams = list(zip(mylist[:-1], mylist[1:]))

In [81]:
#comptage

dd = defaultdict(Counter)
for x, y in bigrams:
    dd[x].update([y])
print(dd)

defaultdict(<class 'collections.Counter'>, {2: Counter({3: 16, 2: 12, 0: 11, 1: 10, 4: 9}), 1: Counter({4: 12, 2: 11, 3: 10, 1: 9, 0: 9}), 0: Counter({0: 13, 3: 13, 2: 12, 4: 11, 1: 7}), 3: Counter({1: 16, 0: 13, 4: 13, 3: 11, 2: 10}), 4: Counter({3: 13, 2: 12, 4: 11, 1: 10, 0: 10})})


In [87]:
#collumnes and lines

max_mylist = max(dd.keys())
all_mylist = list(range(0, max_mylist))
all_mylist

[0, 1, 2, 3, 4]

In [88]:
#transitions

transitions = []
for x in all_mylist:
    row = [dd[x][y] for y in all_mylist]
    transitions.append(row)
transition_table = pd.DataFrame(transitions, index=all_mylist, columns=all_mylist)
transition_table

,0,1,2,3,4
0,13,7,12,13,11
1,9,9,11,10,12
2,11,10,12,16,9
3,13,16,10,11,13
4,10,10,12,13,11


In [89]:
#transforme to probability

transition_table.div(transition_table.sum(axis=1), axis=0)

,0,1,2,3,4
0,0.232143,0.125000,0.214286,0.232143,0.196429
1,0.176471,0.176471,0.215686,0.196078,0.235294
2,0.189655,0.172414,0.206897,0.275862,0.155172
3,0.206349,0.253968,0.158730,0.174603,0.206349
4,0.178571,0.178571,0.214286,0.232143,0.196429


In [118]:
#association whit duration

seq1 = [1/2,1/2,1/4,1/4,'1/8',1/8,1/4]
seq2 = [1/4,'1/4',3/8,1/8,1/2,3/8,1/8]
seq3 = [1/2,'1/4',1/4,1/4,'1/2',1/8,1/8]
seq4 = [1/4,'1/4',1/4,1/2,'1/2',1/4]
seq5 = [1,1/4,1/2,1/4]
seq6 = [1/4,1/8,1/8,1/4,1/4,1/2,1/8,1/8,1/4]
seq7 = [1/2,1/4,1/4,3/4,'1/8',1/8]
seq8 = [1/4,'1/4',1/4,1/4,1/2,'1/4',1/4]
seq9 = ['1/2',1/2,1/4,'1/2',1/4]

#put in array

duration_array =(seq1, seq2, seq3, seq4, seq5)

In [123]:
#lenght and start

chain_length = 4 #nombre de sequence de markov dans la final duration (we can change it)

#don't start with seq1 -> dosn't work 
current_state = seq2
current_indice = 1

final_duration = []

for m in range(chain_length):
    final_duration.append(current_state)
    best_indice = 0
    
    for i in range(0,5) :
        
        #find the best probability
        if transition_table[i][current_indice] > transition_table[best_indice][current_indice] :
            best_indice= i
        
    next_state = duration_array[best_indice]
    current_indice = best_indice
    current_state = next_state
    
print(final_duration)

[[0.25, '1/4', 0.375, 0.125, 0.5, 0.375, 0.125], [1, 0.25, 0.5, 0.25], [0.25, '1/4', 0.25, 0.5, '1/2', 0.25], [0.25, '1/4', 0.375, 0.125, 0.5, 0.375, 0.125]]


- **final duration is a list with the fours sequences chose by the markov model**